In [2]:
import pandas as pd 
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import warnings
import quandl

plt.style.use('seaborn')
# plt.style.use('seaborn-colorblind') #alternative
# plt.rcParams['figure.figsize'] = [16, 9]
plt.rcParams['figure.dpi'] = 300
warnings.simplefilter(action='ignore', category=FutureWarning)

QUANDL_KEY = "zSQgVZXfyBf_ek_efub5"
quandl.ApiConfig.api_key = QUANDL_KEY

In [3]:
df_yahoo = yf.download(['CRM'],
                       start='2018-01-01',
                       end='2022-1-28',
                       #auto_adjust=True,
                       progress=False)

In [4]:
print(f'Downloaded {df_yahoo.shape[0]} rows of data.')
df_yahoo.head()

Downloaded 1026 rows of data.


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,102.879997,104.699997,102.269997,104.410004,104.410004,4669200
2018-01-03,104.900002,106.139999,104.620003,105.290001,105.290001,3888700
2018-01-04,106.000000,107.660004,105.769997,106.680000,106.680000,4540200
2018-01-05,107.760002,108.300003,107.199997,108.099998,108.099998,3368400
2018-01-08,108.000000,109.139999,107.570000,108.860001,108.860001,2933600


In [5]:
df = df_yahoo.loc[:, ['Adj Close']]
df.rename(columns={'Adj Close':'adj_close'}, inplace=True)
df['simple_rtn'] = df.adj_close.pct_change()
df['log_rtn'] = np.log(df.adj_close/df.adj_close.shift(1))
df

,adj_close,simple_rtn,log_rtn
Date,,,
2018-01-02,104.410004,NaN,NaN
2018-01-03,105.290001,0.008428,0.008393
2018-01-04,106.680000,0.013202,0.013115
2018-01-05,108.099998,0.013311,0.013223
2018-01-08,108.860001,0.007031,0.007006
...,...,...,...
2022-01-21,218.630005,-0.019333,-0.019522
2022-01-24,223.029999,0.020125,0.019925
2022-01-25,215.389999,-0.034255,-0.034856


In [6]:
df_quandl = quandl.get(dataset='WIKI/CRM',
                       start_date='2018-01-01', 
                       end_date='2022-1-28')

print(f'Downloaded {df_quandl.shape[0]} rows of data.')
df_quandl.head()

Downloaded 59 rows of data.


,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2018-01-02,102.88,104.700,102.270,104.41,4367522.0,0.0,1.0,102.88,104.700,102.270,104.41,4367522.0
2018-01-03,104.90,106.140,104.620,105.29,3645775.0,0.0,1.0,104.90,106.140,104.620,105.29,3645775.0
2018-01-04,106.00,107.660,105.765,106.68,4480624.0,0.0,1.0,106.00,107.660,105.765,106.68,4480624.0
2018-01-05,107.76,108.295,107.200,108.10,3292050.0,0.0,1.0,107.76,108.295,107.200,108.10,3292050.0
2018-01-08,108.00,109.140,107.570,108.86,2920239.0,0.0,1.0,108.00,109.140,107.570,108.86,2920239.0


In [7]:
df_all_dates = pd.DataFrame(index=pd.date_range(start='2018-01-01', 
                                                end='2022-1-28'))
df = df_all_dates.join(df[['adj_close']], how='left') \
                 .fillna(method='ffill') \
                 .asfreq('M')

In [8]:
df_cpi = quandl.get(dataset='RATEINF/CPI_FRA', 
                    start_date='2018-01-01', 
                    end_date='2022-1-28')
df_cpi.rename(columns={'Value':'cpi'}, inplace=True)

In [9]:
df_merged = df.join(df_cpi, how='left')
df_merged['simple_rtn'] = df_merged.adj_close.pct_change()
df_merged['inflation_rate'] = df_merged.cpi.pct_change()

In [10]:
df_merged

,adj_close,cpi,simple_rtn,inflation_rate
2018-01-31,113.910004,101.75,NaN,NaN
2018-02-28,116.250000,101.72,0.020543,-0.000295
2018-03-31,116.300003,102.75,0.000430,0.010126
2018-04-30,120.989998,102.92,0.040327,0.001655
2018-05-31,129.330002,103.36,0.068931,0.004275
2018-06-30,136.399994,103.37,0.054666,0.000097
2018-07-31,137.149994,103.28,0.005499,-0.000871
2018-08-31,152.679993,103.78,0.113234,0.004841
2018-09-30,159.029999,103.56,0.041590,-0.002120
2018-10-31,137.240005,103.67,-0.137018,0.001062


In [11]:
df_merged['real_rtn'] = (df_merged.simple_rtn + 1) / (df_merged.inflation_rate + 1) - 1
df_merged.head()

,adj_close,cpi,simple_rtn,inflation_rate,real_rtn
2018-01-31,113.910004,101.75,NaN,NaN,NaN
2018-02-28,116.250000,101.72,0.020543,-0.000295,0.020843
2018-03-31,116.300003,102.75,0.000430,0.010126,-0.009599
2018-04-30,120.989998,102.92,0.040327,0.001655,0.038608
2018-05-31,129.330002,103.36,0.068931,0.004275,0.064381


In [12]:
df_yahoo = yf.download(['CRM'   , 'DDD', 'PRLB'],
                       start='2018-01-01',
                       end='2022-1-28',
                       # auto_adjust=True,
                       progress=False)

In [13]:
df_yahoo

Adj Close                              Close             \
                   CRM        DDD        PRLB         CRM        DDD   
Date                                                                   
2018-01-02  104.410004   9.060000  103.849998  104.410004   9.060000   
2018-01-03  105.290001   9.310000  104.250000  105.290001   9.310000   
2018-01-04  106.680000   9.360000  105.349998  106.680000   9.360000   
2018-01-05  108.099998   9.390000  106.250000  108.099998   9.390000   
2018-01-08  108.860001   9.310000  107.349998  108.860001   9.310000   
...                ...        ...         ...         ...        ...   
2022-01-21  218.630005  16.870001   49.349998  218.630005  16.870001   
2022-01-24  223.029999  16.850000   49.130001  223.029999  16.850000   
2022-01-25  215.389999  16.770000   49.119999  215.389999  16.770000   
2022-01-26  211.000000  16.389999   47.759998  211.000000  16.389999   
2022-01-27  212.740005  16.030001   46.580002  212.740005  16.030001   

                              High                                Low  \
                  PRLB         CRM        DDD        PRLB         CRM   
Date                                                                    
2018-01-02  103.849998  104.699997   9.110000  104.250000  102.269997   
2018-01-03  104.250000  106.139999   9.370000  104.550003  104.620003   
2018-01-04  105.349998  107.660004   9.450000  105.379997  105.769997   
2018-01-05  106.250000  108.300003   9.530000  106.589996  107.199997   
2018-01-08  107.349998  109.139999   9.530000  107.550003  107.570000   
...                ...         ...        ...         ...         ...   
2022-01-21   49.349998  224.770004  18.090000   51.220001  218.250000   
2022-01-24   49.130001  223.380005  17.020000   49.340000  207.509995   
2022-01-25   49.119999  223.899994  17.200001   49.490002  214.669998   
2022-01-26   47.759998  221.460007  17.600000   51.490002  209.070007   
2022-01-27   46.580002  223.149994  17.090000   48.360001  212.059998   

                                         Open                         \
                  DDD        PRLB         CRM        DDD        PRLB   
Date                                                                   
2018-01-02   8.600000  102.150002  102.879997   8.700000  103.250000   
2018-01-03   9.100000  103.400002  104.900002   9.120000  103.900002   
2018-01-04   9.090000  104.250000  106.000000   9.350000  104.849998   
2018-01-05   9.220000  105.199997  107.760002   9.420000  105.800003   
2018-01-08   9.210000  104.800003  108.000000   9.380000  105.849998   
...               ...         ...         ...        ...         ...   
2022-01-21  16.870001   49.009998  223.059998  17.990000   49.930000   
2022-01-24  15.330000   46.599998  212.389999  16.160000   48.340000   
2022-01-25  16.190001   47.220001  221.000000  16.320000   48.160000   
2022-01-26  16.270000   47.549999  219.610001  17.440001   49.869999   
2022-01-27  15.880000   46.200001  214.080002  16.730000   48.360001   

              Volume                   
                 CRM      DDD    PRLB  
Date                                   
2018-01-02   4669200  2499000  139700  
2018-01-03   3888700  1799100  128400  
2018-01-04   4540200  1982200  102100  
2018-01-05   3368400  1302700   86700  
2018-01-08   2933600  1898400   84800  
...              ...      ...     ...  
2022-01-21   9116900  3441500  548600  
2022-01-24  11674800  4774900  464700  
2022-01-25   9092700  2552300  308400  
2022-01-26  10029600  1845600  228200  
2022-01-27   9462100  1592700  277900  

[1026 rows x 18 columns]

In [17]:
df = df_yahoo.loc[:, ['Adj Close']]
df.rename(columns={'Adj Close': 'adj_close'}, inplace=True)
df

adj_close                       
                   CRM        DDD        PRLB
Date                                         
2018-01-02  104.410004   9.060000  103.849998
2018-01-03  105.290001   9.310000  104.250000
2018-01-04  106.680000   9.360000  105.349998
2018-01-05  108.099998   9.390000  106.250000
2018-01-08  108.860001   9.310000  107.349998
...                ...        ...         ...
2022-01-21  218.630005  16.870001   49.349998
2022-01-24  223.029999  16.850000   49.130001
2022-01-25  215.389999  16.770000   49.119999
2022-01-26  211.000000  16.389999   47.759998
2022-01-27  212.740005  16.030001   46.580002

[1026 rows x 3 columns]

In [28]:
df = df_all_dates.join(df[['adj_close']], how='left') \
                 .fillna(method='ffill') \
                 .asfreq('M')


KeyError: 'adj_close'